## Exp 1: VGAE using GCNConv

In [86]:
import os.path as osp
import time

import numpy as np
import pickle as pkl
import scipy.sparse as sp
import networkx as nx
import torch

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid, KarateClub
from torch_geometric.nn import GAE, VGAE, DenseGCNConv
from torch_geometric.utils import to_networkx, to_dense_adj, dense_to_sparse

import torch.nn.functional as F
from torch.nn.modules.module import Module
import torch.nn as nn
import torch.nn.modules.loss
from torch.nn.parameter import Parameter
from torch import optim

from sklearn.metrics import roc_auc_score, average_precision_score

In [87]:
def load_dataset(data_dir, dataset_cls, dataset_name):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    transform = T.Compose([
        T.NormalizeFeatures(),
        T.ToDevice(device)
    ])
    obj = dataset_cls(data_dir, dataset_name, transform=transform)
    network = to_networkx(obj[0])
    return nx.adjacency_matrix(network), obj[0].x

def load_dataset_pytorch(data_dir, dataset_cls, dataset_name):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    transform = T.Compose([
        T.NormalizeFeatures(),
        T.ToDevice(device)
    ])
    obj = dataset_cls(data_dir, dataset_name, transform=transform)
    return obj[0].x, obj[0].edge_index

In [57]:
adj, features = load_dataset(
    osp.join('data', 'Planetoid'),
    Planetoid,
    'Cora'
)

# adj, features = load_dataset(
#     osp.join('data', 'KarateClub'),
#     KarateClub,
#     'KarateClub'
# )

In [58]:
adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)

In [85]:
# sp.coo_matrix(adj_train).shape
# train_edges.transpose().shape
# val_edges.transpose().shape
indices, ad = preprocess_graph(adj)
ad.shape

torch.Size([2708, 2708])

In [53]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device)
])
obj = Planetoid(osp.join('data', 'Planetoid'), 'Cora', transform=transform)

In [56]:
to_dense_adj(obj[0].edge_index)[0].shape

torch.Size([2708, 2708])

In [88]:
print("Shape of Adj. Matrix: {}".format(adj.shape))
print("Shape of features: {}".format(tuple(features.shape)))

Shape of Adj. Matrix: (2708, 2708)
Shape of features: (2708, 1433)


In [89]:
class GCNLayer(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, dropout=0., act=F.relu):
        super(GCNLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.act = act
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight)

    def forward(self, input_tensor, adj):
        input_tensor = F.dropout(input_tensor, self.dropout, self.training)
        support = torch.mm(input_tensor, self.weight)
        output_tensor = torch.spmm(adj, support)
        output_tensor = self.act(output_tensor)
        return output_tensor

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [90]:
def loss_function(preds, labels, mu, logvar, n_nodes, norm, pos_weight):
    print(type(pos_weight))
    cost = norm * F.binary_cross_entropy_with_logits(preds, labels, pos_weight=torch.tensor(pos_weight))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 / n_nodes * torch.mean(torch.sum(
        1 + 2 * logvar - mu.pow(2) - logvar.exp().pow(2), 1))
    return cost + KLD

In [91]:
class InnerProductDecoder(Module):
    """Decoder for using inner product for prediction."""

    def __init__(self, dropout, act=torch.sigmoid):
        super(InnerProductDecoder, self).__init__()
        self.dropout = dropout
        self.act = act

    def forward(self, z):
        z = F.dropout(z, self.dropout, training=self.training)
        adj = self.act(torch.mm(z, z.t()))
        return adj

In [140]:
class GCNModelVAE(Module):
    def __init__(self, input_feat_dim, hidden_dim1, hidden_dim2, dropout):
        super(GCNModelVAE, self).__init__()
        self.gc1 = GCNLayer(input_feat_dim, hidden_dim1, dropout, act=F.relu)
        self.gc2 = GCNLayer(hidden_dim1, hidden_dim2, dropout, act=lambda x: x)
        self.gc3 = GCNLayer(hidden_dim1, hidden_dim2, dropout, act=lambda x: x)
        self.dc = InnerProductDecoder(dropout, act=lambda x: x)

    def encode(self, x, adj):
        hidden1 = self.gc1(x, adj)
        return self.gc2(hidden1, adj), self.gc3(hidden1, adj)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x, adj):
        mu, logvar = self.encode(x, adj)
        z = self.reparameterize(mu, logvar)
        return self.dc(z), mu, logvar

In [132]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def preprocess_graph(adj):
    adj = sp.coo_matrix(adj)
    adj_ = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
    # return sparse_to_tuple(adj_normalized)
    return sparse_mx_to_torch_sparse_tensor(adj_normalized)

In [133]:
def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

def mask_test_edges(adj):
    """
    Function to build test set with 10% positive links
    """
    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()
    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0
    
    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    num_test = int(np.floor(edges.shape[0] / 10.))
    num_val = int(np.floor(edges.shape[0] / 20.))
    
    all_edge_idx = list(range(edges.shape[0]))
    np.random.shuffle(all_edge_idx)
    val_edge_idx = all_edge_idx[:num_val]
    test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
    test_edges = edges[test_edge_idx]
    val_edges = edges[val_edge_idx]
    train_edges = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)
    
    def ismember(a, b, tol=5):
        rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
        return np.any(rows_close)
    
    test_edges_false = []
    while len(test_edges_false) < len(test_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], edges_all):
            continue
        if test_edges_false:
            if ismember([idx_j, idx_i], np.array(test_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(test_edges_false)):
                continue
        test_edges_false.append([idx_i, idx_j])
    
    val_edges_false = []
    while len(val_edges_false) < len(val_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], train_edges):
            continue
        if ismember([idx_j, idx_i], train_edges):
            continue
        if ismember([idx_i, idx_j], val_edges):
            continue
        if ismember([idx_j, idx_i], val_edges):
            continue
        if val_edges_false:
            if ismember([idx_j, idx_i], np.array(val_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(val_edges_false)):
                continue
        val_edges_false.append([idx_i, idx_j])
    
    assert ~ismember(test_edges_false, edges_all)
    assert ~ismember(val_edges_false, edges_all)
    assert ~ismember(val_edges, train_edges)
    assert ~ismember(test_edges, train_edges)
    assert ~ismember(val_edges, test_edges)

    data = np.ones(train_edges.shape[0])

    # Re-build adj matrix
    adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
    adj_train = adj_train + adj_train.T

    # NOTE: these edge lists only contain single direction of edge!
    return adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false

def get_roc_score(emb, adj_orig, edges_pos, edges_neg):
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Predict on test set of edges
    adj_rec = np.dot(emb, emb.T)
    preds = []
    pos = []
    for e in edges_pos:
        preds.append(sigmoid(adj_rec[e[0], e[1]]))
        pos.append(adj_orig[e[0], e[1]])

    preds_neg = []
    neg = []
    for e in edges_neg:
        preds_neg.append(sigmoid(adj_rec[e[0], e[1]]))
        neg.append(adj_orig[e[0], e[1]])

    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)

    return roc_score, ap_score

In [134]:
args = {
    'seed' : 42, 
    'epochs' : 200,
    'hidden1' : 64,
    'hidden2' : 32,
    'lr' : 0.05,
    'dropout' : 0.,
    'dataset_str' : 'Cora'
}

In [138]:
def build_model(args):
    print("Using {} dataset".format(args['dataset_str']))
    adj, features = load_dataset(
        osp.join('data', 'Planetoid'),
        Planetoid,
        args['dataset_str']
    )
    # adj, features = load_dataset(
    #     osp.join('data', 'KarateClub'),
    #     KarateClub,
    #     'KarateClub'
    # )
    n_nodes, feat_dim = tuple(features.shape)

    # Store original adjacency matrix (without diagonal entries) for later
    adj_orig = adj
    adj_orig = adj_orig - sp.dia_matrix((adj_orig.diagonal()[np.newaxis, :], [0]), shape=adj_orig.shape)
    adj_orig.eliminate_zeros()
    
    adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)
    adj = adj_train
    
    # Some preprocessing
    adj_norm = preprocess_graph(adj)
    adj_label = adj_train + sp.eye(adj_train.shape[0])
    # adj_label = sparse_to_tuple(adj_label)
    adj_label = torch.FloatTensor(adj_label.toarray())
    
    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)
    
    model = GCNModelVAE(feat_dim, args['hidden1'], args['hidden2'], args['dropout'])
    optimizer = optim.Adam(model.parameters(), lr=args['lr'])
    
    hidden_emb = None
    # print(torch.unsqueeze(features, 0).shape, torch.unsqueeze(adj_norm, 0).shape)
    for epoch in range(args['epochs']):
        t = time.time()
        model.train()
        optimizer.zero_grad()
        recovered, mu, logvar = model(features, adj_norm)
        loss = loss_function(preds=recovered, labels=adj_label,
                             mu=mu, logvar=logvar, n_nodes=n_nodes,
                             norm=norm, pos_weight=pos_weight)
        loss.backward()
        cur_loss = loss.item()
        optimizer.step()

        hidden_emb = mu.data.numpy()
        roc_curr, ap_curr = get_roc_score(hidden_emb, adj_orig, val_edges, val_edges_false)

        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cur_loss),
              "val_ap=", "{:.5f}".format(ap_curr),
              "time=", "{:.5f}".format(time.time() - t)
              )
    
    print("Optimization Finished!")

    roc_score, ap_score = get_roc_score(hidden_emb, adj_orig, test_edges, test_edges_false)
    print('Test ROC score: ' + str(roc_score))
    print('Test AP score: ' + str(ap_score))
    
    return model

In [139]:
model = build_model(args)

Using Cora dataset
<class 'numpy.float64'>
Epoch: 0001 train_loss= 2.31319 val_ap= 0.74717 time= 0.17375
<class 'numpy.float64'>
Epoch: 0002 train_loss= 1.97769 val_ap= 0.71301 time= 0.10004
<class 'numpy.float64'>
Epoch: 0003 train_loss= 1.46324 val_ap= 0.71035 time= 0.09716
<class 'numpy.float64'>
Epoch: 0004 train_loss= 1.00397 val_ap= 0.71371 time= 0.09928
<class 'numpy.float64'>
Epoch: 0005 train_loss= 0.76198 val_ap= 0.71753 time= 0.09559
<class 'numpy.float64'>
Epoch: 0006 train_loss= 0.76545 val_ap= 0.70523 time= 0.08602
<class 'numpy.float64'>
Epoch: 0007 train_loss= 0.78406 val_ap= 0.70346 time= 0.08942
<class 'numpy.float64'>
Epoch: 0008 train_loss= 0.76408 val_ap= 0.71239 time= 0.09347
<class 'numpy.float64'>
Epoch: 0009 train_loss= 0.83149 val_ap= 0.71608 time= 0.09229
<class 'numpy.float64'>
Epoch: 0010 train_loss= 0.77901 val_ap= 0.72547 time= 0.09052
<class 'numpy.float64'>
Epoch: 0011 train_loss= 0.80099 val_ap= 0.72192 time= 0.08794
<class 'numpy.float64'>
Epoch: 0012

## Exp 2: VGAE using ChebConv

In [137]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn.models import InnerProductDecoder, VGAE
from torch_geometric.nn.conv import GCNConv, ChebConv, SAGEConv, GraphConv, ResGatedGraphConv, GATConv, GATv2Conv
from torch_geometric.utils import negative_sampling, remove_self_loops, add_self_loops

import os
from torch.optim import Adam

from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges

In [138]:
class GCNEncoder(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.gcn_shared = GATv2Conv(in_channels, hidden_channels)
        self.gcn_mu = GATv2Conv(hidden_channels, out_channels)
        self.gcn_logvar = GATv2Conv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.gcn_shared(x, edge_index))
        mu = self.gcn_mu(x, edge_index)
        logvar = self.gcn_logvar(x, edge_index)
        return mu, logvar


class DeepVGAE(VGAE):
    def __init__(self, args):
        super(DeepVGAE, self).__init__(encoder=GCNEncoder(args["enc_in_channels"],
                                                          args["enc_hidden_channels"],
                                                          args["enc_out_channels"]),
                                       decoder=InnerProductDecoder())

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        adj_pred = self.decoder.forward_all(z)
        return adj_pred

    def loss(self, x, pos_edge_index, all_edge_index):
        z = self.encode(x, pos_edge_index)

        pos_loss = -torch.log(
            self.decoder(z, pos_edge_index, sigmoid=True) + 1e-15).mean()

        # Do not include self-loops in negative samples
        all_edge_index_tmp, _ = remove_self_loops(all_edge_index)
        all_edge_index_tmp, _ = add_self_loops(all_edge_index_tmp)

        neg_edge_index = negative_sampling(all_edge_index_tmp, z.size(0), pos_edge_index.size(1))
        neg_loss = -torch.log(1 - self.decoder(z, neg_edge_index, sigmoid=True) + 1e-15).mean()

        kl_loss = 1 / x.size(0) * self.kl_loss()

        return pos_loss + neg_loss + kl_loss

    def single_test(self, x, train_pos_edge_index, test_pos_edge_index, test_neg_edge_index):
        with torch.no_grad():
            z = self.encode(x, train_pos_edge_index)
        roc_auc_score, average_precision_score = self.test(z, test_pos_edge_index, test_neg_edge_index)
        return roc_auc_score, average_precision_score

In [143]:
args = {
    "dataset": "Cora",
    "enc_in_channels": 1433,
    "enc_hidden_channels": 128,
    "enc_out_channels": 64,
    "lr": 0.007,
    "epoch": 200
}

In [144]:
torch.manual_seed(12345)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [145]:
model = DeepVGAE(args).to(device)
optimizer = Adam(model.parameters(), lr=args["lr"])

os.makedirs("datasets", exist_ok=True)
dataset = Planetoid(os.path.join('data', 'Planetoid'), args["dataset"], transform=T.NormalizeFeatures())
data = dataset[0].to(device)
all_edge_index = data.edge_index
data = train_test_split_edges(data, 0.05, 0.1)

/Users/ashutosh1919/miniforge3/envs/persona2vec/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [146]:
for epoch in range(args["epoch"]):
    model.train()
    optimizer.zero_grad()
    loss = model.loss(data.x, data.train_pos_edge_index, all_edge_index)
    loss.backward()
    optimizer.step()
    if epoch % 1 == 0:
        model.eval()
        roc_auc, ap = model.single_test(data.x,
                                        data.train_pos_edge_index,
                                        data.test_pos_edge_index,
                                        data.test_neg_edge_index)
        print("Epoch {} - Loss: {} ROC_AUC: {} Precision: {}".format(epoch, loss.cpu().item(), roc_auc, ap))

Epoch 0 - Loss: 7.020147323608398 ROC_AUC: 0.512834453730075 Precision: 0.5619028995951968
Epoch 1 - Loss: 6.353734016418457 ROC_AUC: 0.5404477026165795 Precision: 0.5688020983381454
Epoch 2 - Loss: 5.948556423187256 ROC_AUC: 0.5545135725833457 Precision: 0.5685748986836247
Epoch 3 - Loss: 5.289628505706787 ROC_AUC: 0.5640012386175012 Precision: 0.5644605540367131
Epoch 4 - Loss: 4.593630790710449 ROC_AUC: 0.5601827680940773 Precision: 0.5574427438826639
Epoch 5 - Loss: 3.9499638080596924 ROC_AUC: 0.5565137238099009 Precision: 0.5543630786466682
Epoch 6 - Loss: 3.3577821254730225 ROC_AUC: 0.5451429270979984 Precision: 0.5463557489838422
Epoch 7 - Loss: 2.827617883682251 ROC_AUC: 0.5265132557277058 Precision: 0.526604150797414
Epoch 8 - Loss: 2.3036341667175293 ROC_AUC: 0.49540379290603426 Precision: 0.5060980026865113
Epoch 9 - Loss: 1.9329620599746704 ROC_AUC: 0.4918355663254468 Precision: 0.50902803072839
Epoch 10 - Loss: 1.7288002967834473 ROC_AUC: 0.5130792967245048 Precision: 0.52